In [2]:
# Core libs
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

In [3]:
SIMPLE_COLUMNS = [
    'Fecha',
    'Calle',
    'Piso',
    'Departamento',
    'Provincia',
    'Distrito',
    'Estacionamiento',
    'Depositos',
    'Lat',
    'Lon',
    'Categoria',
    'Posicion',
    'Frentes',
    'Edad',
    'Elevador',
    'Conservacion',
    'Metodo',
    'Moneda',
    'AreaTerreno',
    'AreaConstruccion',
    'PromedioConstruccion',
    'PromedioTerreno',
    'Valorcomercial'
]

NUM_COLS = [
    'Fecha entrega del Informe', 
    'Tipo de vía',
    'Número de estacionamiento', #aug
    'Depósitos', #aug
    'Latitud (Decimal)', 
    'Longitud (Decimal)',
    'Número de frentes',
    'Edad', #aug
    'Elevador', 
    'Área Terreno', #aug
    'Área Construcción', #aug
    'Valor comercial (USD)',
]

CAT_COLS = [
    'Calle',
    'Piso', #irrelevant
    'Departamento', 
    'Provincia',  
    'Distrito',  
    'Categoría del bien', 
    'Posición', #irrelevant
    'Estado de conservación', 
    'Método Representado', #irrelevant
    'Moneda principal para cálculos'
]

TO_AUGMENT = [
    'Número de estacionamiento',
    'Depósitos',
    'Edad'
]

In [4]:
def string_convert(x):
    try:
        return float(x.replace(',',''))
    except ValueError:
        return 0.0

In [5]:
def data_sanitize_train(raw_data, train = False, region = None):

    if region is not None:
        ordinal_data = raw_data[raw_data[region[0]] == region[1]].copy()
    else:
        ordinal_data = raw_data.copy()

    ranva = pd.Series(np.random.randn(ordinal_data.shape[0]))
    ordinal_data = ordinal_data.assign(ranva=ranva.values)

    for column in raw_data[:-2]:
        dist_encod = OrdinalEncoder()

        if column in CAT_COLS:
            enc = dist_encod.fit_transform(ordinal_data[[column,'ranva']].to_numpy(dtype = str))
            ordinal_data[column] = enc.T[0]

    ordinal_data = ordinal_data.drop('ranva', axis = 1)

    ordinal_data = ordinal_data.applymap(lambda x : string_convert(x) if isinstance(x,str) else x)
    ordinal_data.columns = SIMPLE_COLUMNS
    scale_data = ordinal_data.applymap(lambda x : np.cbrt(x))

    if train:
        ground_truth = ordinal_data['Valorcomercial'].to_numpy()
        return scale_data, ground_truth
    
    return scale_data

def data_sanitize_test(raw_data, augment, region=None):
    augmented_data = raw_data.copy()

    ages = augmented_data['Edad'].to_numpy()
    dates = augmented_data['Fecha entrega del Informe'].to_numpy()

    for i, age in enumerate(ages):
        if age >= 1000:
            ages[i] = np.round(((dates[i] - pd.Timestamp(ages[i])).days)/365.0)

    augmented_data['Edad'] = ages

    augmented_data.insert(augmented_data.shape[1]-2, 'Precio promedio m2 (Terreno)', np.zeros(augmented_data.shape[0]))
    augmented_data.insert(augmented_data.shape[1]-2, 'Precio promedio m2 (Construcción)', np.zeros(augmented_data.shape[0]))

    augmented_data['Área Terreno'] = augmented_data['Área Terreno'].fillna(0.0)
    augmented_data['Área Construcción'] = augmented_data['Área Construcción'].fillna(0.0)

    for dep in raw_data['Departamento'].unique():

        dep_select = raw_data[raw_data['Departamento'] == dep]
        for prov in dep_select['Provincia'].unique():

            prov_select = dep_select[dep_select['Provincia'] == prov]
            for dist in prov_select['Distrito'].unique():
                query = raw_data[(raw_data['Departamento'] == dep) &
                                 (raw_data['Provincia'] == prov  ) &
                                 (raw_data['Distrito'] == dist   )]

                fault_key = False

                for augment_dim in TO_AUGMENT:
                    to_augment_array = np.array([val.replace(',','') if isinstance(val, str) else val for val in query[augment_dim].to_list()], dtype=np.float32)

                    try:
                        to_augment_array = np.nan_to_num(to_augment_array, nan = augment[(dep,prov,dist)][augment_dim])
                    except KeyError:
                        to_augment_array = np.nan_to_num(to_augment_array, nan = 0.0)
                        fault_key = True

                    full_array = augmented_data[augment_dim].to_numpy()                    
                    full_array[query.index] = to_augment_array

                    augmented_data[augment_dim] = full_array

                if fault_key: continue
                
                avg_bval_list = augmented_data['Precio promedio m2 (Construcción)'].to_numpy()
                avg_pval_list = augmented_data['Precio promedio m2 (Terreno)'].to_numpy()
                avg_bval_list[query.index] = augment[(dep,prov,dist)]['Precio promedio m2 (Construcción)']
                avg_pval_list[query.index] = augment[(dep,prov,dist)]['Precio promedio m2 (Terreno)']
                augmented_data['Precio promedio m2 (Construcción)'] = avg_bval_list
                augmented_data['Precio promedio m2 (Terreno)'] = avg_pval_list

    return data_sanitize_train(augmented_data.applymap(lambda x : (x - pd.Timestamp("1900-01-01 00:00:00")).days + 2 if isinstance(x,pd.Timestamp) else x).fillna(-1), train=True, region=region)

In [6]:
DATA_TRAIN_FILENAME_XLXS = '../data/xlsx/newdata_augmented.xlsx'
DATA_TEST_FILENAME_XLXS = '../data/xlsx/newdata.xlsx'
DATA_AUGMENTER_FILENAME = '../data/xlsx/newdata_augmentation.pkl'

In [7]:
augmenter = pickle.load(open(DATA_AUGMENTER_FILENAME, "rb"))
raw_data_master = pd.read_excel(DATA_TRAIN_FILENAME_XLXS, header = [0]).applymap(lambda x : (x - pd.Timestamp("1900-01-01 00:00:00")).days + 2 if isinstance(x,pd.Timestamp) else x).fillna(-1)
raw_test_master = pd.read_excel(DATA_TEST_FILENAME_XLXS, header = [0])

raw_test_master = raw_test_master.sample(frac=0.2)

raw_data_master = raw_data_master.drop(index=list(raw_test_master.index))

raw_test_master.index = list(range(raw_test_master.shape[0]))
raw_data_master.index = list(range(raw_data_master.shape[0]))

raw_data_master = raw_data_master.applymap(lambda x : (x - pd.Timestamp("1900-01-01 00:00:00")).days + 2 if isinstance(x,pd.Timestamp) else x).fillna(-1)

In [8]:
#trainer_data, train_gt = data_sanitize_train(raw_data_master, True, ('Provincia', 'Lima'))
#tester_data, test_gt = data_sanitize_test(raw_test_master, augmenter, ('Provincia', 'Lima'))


trainer_data, train_gt = data_sanitize_train(raw_data_master, True)
tester_data, test_gt = data_sanitize_test(raw_test_master, augmenter)

In [9]:
trainer_data

,Fecha,Calle,Piso,Departamento,Provincia,Distrito,Estacionamiento,Depositos,Lat,Lon,...,Edad,Elevador,Conservacion,Metodo,Moneda,AreaTerreno,AreaConstruccion,PromedioConstruccion,PromedioTerreno,Valorcomercial
0,34.928835,20.906615,1.0,2.410142,4.0,2.843867,1.0,0.0,-2.293209,-4.255179,...,1.587401,-1.0,0.000000,1.000000,0.0,2.515109,20.580489,10.626586,4.026902,43.752397
1,35.215272,21.682067,1.0,2.410142,4.0,5.180101,1.0,0.0,-2.295125,-4.255044,...,0.000000,-1.0,1.587401,1.000000,0.0,1.955692,25.044720,12.016182,3.753922,48.148125
2,35.061389,19.720273,1.0,2.410142,4.0,6.679420,1.0,0.0,-2.300885,-4.252988,...,2.668402,-1.0,1.000000,1.259921,0.0,6.214465,15.105917,9.016431,8.271904,67.327203
3,35.363024,6.882389,1.0,2.410142,4.0,5.905941,0.0,0.0,-2.294081,-4.255566,...,1.000000,-1.0,1.817121,1.000000,0.0,1.759809,22.679875,11.462385,3.735380,43.136039
4,34.880134,5.635741,1.0,2.410142,4.0,5.167649,1.0,1.0,-2.296238,-4.254582,...,2.714418,-1.0,0.000000,1.000000,0.0,4.422031,20.255067,12.489057,5.371685,66.197754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31690,35.154420,8.675697,1.0,2.410142,4.0,6.518684,0.0,0.0,-2.296571,-4.254589,...,0.000000,-1.0,1.442250,1.000000,0.0,1.967818,24.585154,11.937172,3.771814,43.650992
31691,34.903954,20.387446,1.0,2.410142,4.0,6.423158,1.0,1.0,-2.297914,-4.254385,...,1.817121,-1.0,0.000000,1.000000,0.0,3.760298,19.944848,12.224924,6.048314,71.928105
31692,35.385667,8.853599,1.0,2.410142,4.0,5.277632,0.0,0.0,-2.295575,-4.255425,...,1.442250,-1.0,1.000000,1.000000,0.0,2.223980,24.648961,11.913733,4.431048,53.090388
31693,35.390724,6.082202,1.0,2.410142,4.0,6.179747,0.0,0.0,-2.295810,-4.254606,...,3.684031,-1.0,1.000000,1.259921,0.0,7.306144,24.068094,13.043248,6.630044,93.527354
